# 🚀 Complete Colab Development Environment

## AI Coding Assistant - Full Cloud Development Setup

This notebook provides a complete development environment for your AI coding assistant entirely within Google Colab. No local setup required!

### 🎯 What This Environment Includes

- **🔧 Full Backend API**: FastAPI server with all endpoints
- **🤖 AI Services**: Multiple AI model integrations
- **💾 Database**: SQLite with full schema
- **🌐 Web Interface**: Streamlit-based frontend
- **📊 Model Training**: Custom model fine-tuning capabilities
- **🔄 Real-time Updates**: Live code editing and testing
- **📱 Mobile Access**: Access from any device

### 🚀 Quick Start

1. **Run Setup**: Execute the setup cells below
2. **Start Services**: Launch backend and frontend
3. **Access Interface**: Use the provided URLs
4. **Start Coding**: Begin using your AI assistant!

---


In [ ]:
# 🔧 Environment Setup
print('🚀 Setting up complete Colab development environment...')

import os
import sys
import subprocess
import json
from pathlib import Path

# Install required packages
packages = [
    'fastapi',
    'uvicorn[standard]',
    'streamlit',
    'sqlalchemy',
    'alembic',
    'python-multipart',
    'python-jose[cryptography]',
    'passlib[bcrypt]',
    'transformers',
    'torch',
    'accelerate',
    'datasets',
    'peft',
    'bitsandbytes',
    'openai',
    'anthropic',
    'requests',
    'websockets',
    'pyngrok',
    'python-dotenv'
]

print('📦 Installing packages...')
for package in packages:
    !pip install -q {package}

print('✅ Package installation complete!')

# Create project structure
project_root = '/content/ai_assistant'
os.makedirs(project_root, exist_ok=True)
os.chdir(project_root)

# Create directory structure
directories = [
    'backend/app/api',
    'backend/app/core',
    'backend/app/models',
    'backend/app/services',
    'backend/app/utils',
    'frontend',
    'data/training',
    'data/models',
    'logs',
    'config'
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)

print(f'📁 Project structure created at: {project_root}')
print('🎉 Environment setup complete!')

## 🔧 Backend Setup

Setting up the complete FastAPI backend with all services and endpoints.


In [ ]:
# 🔧 Backend Core Configuration

# Create main.py
main_py = '''
from fastapi import FastAPI, HTTPException, Depends
from fastapi.middleware.cors import CORSMiddleware
from fastapi.security import HTTPBearer
import uvicorn
import os

from app.api import chat, code, auth, models
from app.core.config import settings
from app.core.database import engine, Base

# Create database tables
Base.metadata.create_all(bind=engine)

app = FastAPI(
    title="AI Coding Assistant API",
    description="Complete AI-powered coding assistant",
    version="2.0.0"
)

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Include routers
app.include_router(auth.router, prefix="/api/auth", tags=["auth"])
app.include_router(chat.router, prefix="/api/chat", tags=["chat"])
app.include_router(code.router, prefix="/api/code", tags=["code"])
app.include_router(models.router, prefix="/api/models", tags=["models"])

@app.get("/")
async def root():
    return {"message": "AI Coding Assistant API - Running in Colab!"}

@app.get("/health")
async def health_check():
    return {"status": "healthy", "environment": "colab"}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

with open('backend/app/main.py', 'w') as f:
    f.write(main_py.strip())

# Create __init__.py files
init_files = [
    'backend/__init__.py',
    'backend/app/__init__.py',
    'backend/app/api/__init__.py',
    'backend/app/core/__init__.py',
    'backend/app/models/__init__.py',
    'backend/app/services/__init__.py',
    'backend/app/utils/__init__.py'
]

for init_file in init_files:
    with open(init_file, 'w') as f:
        f.write('')

print('✅ Backend core files created!')

In [ ]:
# 🔧 Configuration Files

# Create config.py
config_py = '''
import os
from typing import Optional

class Settings:
    # Database
    DATABASE_URL: str = "sqlite:///./ai_assistant.db"
    
    # Security
    SECRET_KEY: str = "colab-ai-assistant-secret-key-2024"
    ALGORITHM: str = "HS256"
    ACCESS_TOKEN_EXPIRE_MINUTES: int = 30
    
    # AI Services
    OPENAI_API_KEY: Optional[str] = os.getenv("OPENAI_API_KEY")
    ANTHROPIC_API_KEY: Optional[str] = os.getenv("ANTHROPIC_API_KEY")
    
    # Model Settings
    DEFAULT_MODEL: str = "gpt-3.5-turbo"
    LOCAL_MODEL_PATH: str = "/content/ai_assistant/data/models"
    
    # Colab Settings
    COLAB_MODE: bool = True
    NGROK_TOKEN: Optional[str] = os.getenv("NGROK_TOKEN")

settings = Settings()
'''

with open('backend/app/core/config.py', 'w') as f:
    f.write(config_py.strip())

# Create database.py
database_py = '''
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from .config import settings

engine = create_engine(settings.DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
'''

with open('backend/app/core/database.py', 'w') as f:
    f.write(database_py.strip())

print('✅ Configuration files created!')

In [ ]:
# 🗄️ Database Models

# Create user.py
user_py = '''
from sqlalchemy import Column, Integer, String, DateTime, Boolean
from sqlalchemy.sql import func
from ..core.database import Base

class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True, index=True)
    email = Column(String, unique=True, index=True)
    hashed_password = Column(String)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime(timezone=True), server_default=func.now())
'''

with open('backend/app/models/user.py', 'w') as f:
    f.write(user_py.strip())

# Create chat.py
chat_py = '''
from sqlalchemy import Column, Integer, String, Text, DateTime, ForeignKey
from sqlalchemy.sql import func
from sqlalchemy.orm import relationship
from ..core.database import Base

class ChatSession(Base):
    __tablename__ = "chat_sessions"
    
    id = Column(Integer, primary_key=True, index=True)
    user_id = Column(Integer, ForeignKey("users.id"))
    title = Column(String, default="New Chat")
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    
    messages = relationship("ChatMessage", back_populates="session")

class ChatMessage(Base):
    __tablename__ = "chat_messages"
    
    id = Column(Integer, primary_key=True, index=True)
    session_id = Column(Integer, ForeignKey("chat_sessions.id"))
    role = Column(String)  # user, assistant, system
    content = Column(Text)
    timestamp = Column(DateTime(timezone=True), server_default=func.now())
    
    session = relationship("ChatSession", back_populates="messages")
'''

with open('backend/app/models/chat.py', 'w') as f:
    f.write(chat_py.strip())

print('✅ Database models created!')

In [ ]:
# 🤖 AI Service Implementation

ai_service_py = '''
import openai
import anthropic
from typing import Optional, List, Dict, Any
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import os

from ..core.config import settings

class ColabAIService:
    def __init__(self):
        self.openai_client = None
        self.anthropic_client = None
        self.local_model = None
        self.local_tokenizer = None
        self.setup_clients()
    
    def setup_clients(self):
        """Initialize AI service clients"""
        if settings.OPENAI_API_KEY:
            self.openai_client = openai.OpenAI(api_key=settings.OPENAI_API_KEY)
        
        if settings.ANTHROPIC_API_KEY:
            self.anthropic_client = anthropic.Anthropic(api_key=settings.ANTHROPIC_API_KEY)
    
    async def load_local_model(self, model_name: str = "microsoft/DialoGPT-medium"):
        """Load a local model for inference"""
        try:
            print(f"Loading local model: {model_name}")
            self.local_tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.local_model = AutoModelForCausalLM.from_pretrained(model_name)
            
            if torch.cuda.is_available():
                self.local_model = self.local_model.cuda()
                print("✅ Model loaded on GPU")
            else:
                print("✅ Model loaded on CPU")
            
            return True
        except Exception as e:
            print(f"❌ Error loading local model: {e}")
            return False
    
    async def generate_code(self, prompt: str, language: str = "python", model: str = None) -> Dict[str, Any]:
        """Generate code using available AI services"""
        try:
            # Try OpenAI first
            if self.openai_client:
                return await self._generate_with_openai(prompt, language, model)
            
            # Fallback to Anthropic
            if self.anthropic_client:
                return await self._generate_with_anthropic(prompt, language)
            
            # Fallback to local model
            if self.local_model:
                return await self._generate_with_local(prompt, language)
            
            return {"error": "No AI services available"}
        
        except Exception as e:
            return {"error": f"Code generation failed: {str(e)}"}
    
    async def _generate_with_openai(self, prompt: str, language: str, model: str = None) -> Dict[str, Any]:
        model = model or settings.DEFAULT_MODEL
        
        system_prompt = f"""You are an expert {language} programmer. Generate clean, efficient, and well-commented code based on the user's request."""
        
        response = self.openai_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=2000
        )
        
        return {
            "code": response.choices[0].message.content,
            "model": model,
            "service": "openai"
        }
    
    async def _generate_with_anthropic(self, prompt: str, language: str) -> Dict[str, Any]:
        system_prompt = f"You are an expert {language} programmer. Generate clean, efficient, and well-commented code."
        
        response = self.anthropic_client.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=2000,
            system=system_prompt,
            messages=[{"role": "user", "content": prompt}]
        )
        
        return {
            "code": response.content[0].text,
            "model": "claude-3-sonnet",
            "service": "anthropic"
        }
    
    async def _generate_with_local(self, prompt: str, language: str) -> Dict[str, Any]:
        # Simple local generation (placeholder)
        inputs = self.local_tokenizer.encode(prompt, return_tensors='pt')
        
        if torch.cuda.is_available():
            inputs = inputs.cuda()
        
        with torch.no_grad():
            outputs = self.local_model.generate(
                inputs,
                max_length=inputs.shape[1] + 200,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.local_tokenizer.eos_token_id
            )
        
        generated_text = self.local_tokenizer.decode(outputs[0], skip_special_tokens=True)
        code = generated_text[len(prompt):].strip()
        
        return {
            "code": code,
            "model": "local",
            "service": "local"
        }
    
    def get_status(self) -> Dict[str, Any]:
        """Get service status"""
        return {
            "openai_available": self.openai_client is not None,
            "anthropic_available": self.anthropic_client is not None,
            "local_model_loaded": self.local_model is not None,
            "gpu_available": torch.cuda.is_available(),
            "environment": "colab"
        }
'''

with open('backend/app/services/ai_service.py', 'w') as f:
    f.write(ai_service_py.strip())

print('✅ AI service implementation created!')